Представьте, что вы работаете DS в компании TripAdvisor. Одна из проблем компании — это нечестные рестораны, которые накручивают себе рейтинг. Одним из способов нахождения таких ресторанов является построение модели, которая предсказывает рейтинг ресторана. Если предсказания модели сильно отличаются от фактического результата, то, возможно, ресторан играет нечестно, и его стоит проверить.

Вам поставлена задача создать такую модель. Готовы приступить?

Первоначальная версия датасета состоит из десяти столбцов, содержащих следующую информацию:

1. Restaurant_id — идентификационный номер ресторана / сети ресторанов;
2. City — город, в котором находится ресторан;
3. Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
4. Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
5. Rating — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
6. Price Range — диапазон цен в ресторане;
7. Number of Reviews — количество отзывов о ресторане;
8. Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
9. URL_TA — URL страницы ресторана на TripAdvosor;
10. ID_TA — идентификатор ресторана в базе данных TripAdvisor.

In [ ]:
import numpy as np 
import pandas as pd 
from datetime import datetime, date, time

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

In [ ]:
pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)
df_train = pd.read_csv('main_task.csv')
df_test = pd.read_csv('kaggle_task.csv')
sample_submission = pd.read_csv('sample_submission.csv')
# промаркируем тренировочный и тестовый сеты

df_train['mark'] = 1
df_test['mark'] = 0

# пропущенные значения для тестового сета заменим на 0
df_test['Rating'] = 0  

# объединим сеты для общей обработки данных
df = df_test.append(df_train, sort=False).reset_index(drop=True)
df.head()

In [ ]:
# проверим наименования столбцов и заменим их на более краткие
df.columns

Назначим наименования столбцов для своего удобства

In [ ]:
df.columns = ['id','city','cuisine','ranking','price','num_rew',
             'reviews','url','id_ta','mark','rating']

Проверим столбцы на пропуски в данных

In [ ]:
df.info()

В части столбцов есть пропуски. Суммарное количество строк 50000.

# Проверим столбцы отдельно.

# Restraunt_id

In [ ]:
df.id.isna().value_counts()

Пропусков нет, но тип данных не числовой. Преобразуем, удалив id_ и преобразовав в INT

In [ ]:
df['id']=df.id.apply(lambda x: int(x[3:]))

In [ ]:
# df.head()#проверим изменения в столбце

In [ ]:
# проверим длинну, чтоб убедиться в отсутствии потери данных
len(df.id)


# Проверим столбец City

In [ ]:
df.city.isna().value_counts()

Проверим правильность записи наименований городов

In [ ]:
# df.city.value_counts()

In [ ]:
df.city.describe()

Согласно описанию солбца уникальных городов 31, Лондон наиболее часто упоминаемый

Создадим словарь, содержащий наименования городов в качестве ключей и порядковый номер, в качестве числового признака

In [ ]:
A = list(df.city.value_counts().keys())
B = range(0, len(A))
dict_city = dict(zip(A, B))
dict_city

Добавим в набор данных признак, основанный на порядковом номере города

In [ ]:
df['City_ind'] = df['city'].replace(A, B)

Приведем отзывы к количесту упоминаний городов

In [ ]:
A = list(df.city.value_counts().keys())
df_city = pd.DataFrame()
df_city['city'] = df['city']
df_city['ranking'] = df['ranking']
df_city = pd.DataFrame(df_city.groupby(['city']).max())
df_rank = df_city['ranking']

A = list(df_rank.keys())
B = list(df_rank)

df['city_rest'] = df['city'].replace(A, B)

df['std_rank'] = df['ranking']/df['city_rest']

df.head()
A

На основании данных переписи населения можем составить дополнительный числовой признак по количеству населения.

Для этого созададим список с жителями, в соответствии с порядковым номером города

P.S. В baseline приведен список с население, который сводится со списком городов. В итоге в списке А города по алфавиту и население некорректно по ним распределяется. Здесь население указано в соответствии с фактической последовательностью городов при загрузке их в список

In [ ]:

B = [869709, 664046, 5575000, 3769000, 424428, 174383, 1752000, 602481,
    1388000, 482005, 499480, 1899000, 631695, 769498, 504718, 342039,
    8982000, 613894, 513275, 6642000, 1352000, 1472000, 214349, 681067,
    2148000, 1309000, 2873000, 975904, 1897000, 1708000, 402762]
dict_popul = dict(zip(A,B))
df['population'] = df['city'].replace(A,B)
df.head()


По аналогии с городами добавим числовые признаки стран. Не будем выдумывать и воспользуемся кодом из Baseline

Для понимания: присвоим новому столбцу строковые значения столбца city, потом заменим их, составим словарь с ключами-странами и значениями-индексами. Далее в новом столбце Country заменим наименования стран на числовые значения.

In [ ]:
df['country'] = df.city
df['country'] = df['country'].replace('London', 'GreatBritain')
df['country'] = df['country'].replace('Paris', 'France')
df['country'] = df['country'].replace('Madrid', 'Spain')
df['country'] = df['country'].replace('Barcelona', 'Spain')
df['country'] = df['country'].replace('Berlin', 'Germany')
df['country'] = df['country'].replace('Milan', 'Italy')
df['country'] = df['country'].replace('Rome', 'Italy')
df['country'] = df['country'].replace('Prague', 'CzechRepublic')
df['country'] = df['country'].replace('Lisbon', 'Portugal')
df['country'] = df['country'].replace('Vienna', 'Austria')
df['country'] = df['country'].replace('Amsterdam', 'Netherlands')
df['country'] = df['country'].replace('Brussels', 'Belgium')
df['country'] = df['country'].replace('Hamburg', 'Germany')
df['country'] = df['country'].replace('Munich', 'Germany')
df['country'] = df['country'].replace('Lyon', 'France')
df['country'] = df['country'].replace('Stockholm', 'Sweden')
df['country'] = df['country'].replace('Budapest', 'Hungary')
df['country'] = df['country'].replace('Warsaw', 'Poland')
df['country'] = df['country'].replace('Dublin', 'Irland')
df['country'] = df['country'].replace('Copenhagen', 'Denmark')
df['country'] = df['country'].replace('Athens', 'Greece')
df['country'] = df['country'].replace('Edinburgh', 'Scotland')
df['country'] = df['country'].replace('Zurich', 'Switzeland')
df['country'] = df['country'].replace('Oporto', 'Portugal')
df['country'] = df['country'].replace('Geneva', 'Switzeland')
df['country'] = df['country'].replace('Krakow', 'Poland')
df['country'] = df['country'].replace('Oslo', 'Norway')
df['country'] = df['country'].replace('Helsinki', 'Finland')
df['country'] = df['country'].replace('Bratislava', 'Slovakia')
df['country'] = df['country'].replace('Luxembourg', 'Luxembourg')
df['country'] = df['country'].replace('Ljubljana', 'Slovenia')

A = list(df.country.value_counts().keys())
B = range(0, len(A))
dict_country = dict(zip(A, B))
# словарь со значениями стран


df['country_ind'] = df['country'].replace(A, B)

Добавим показатель турпотока в страны Европы. Возьмем статистические данные с открытых источников
https://knoema.ru/atlas/topics/%d0%a2%d1%83%d1%80%d0%b8%d0%b7%d0%bc/%d0%9a%d0%bb%d1%8e%d1%87%d0%b5%d0%b2%d1%8b%d0%b5-%d0%bf%d0%be%d0%ba%d0%b0%d0%b7%d0%b0%d1%82%d0%b5%d0%bb%d0%b8-%d1%82%d1%83%d1%80%d0%b8%d0%b7%d0%bc%d0%b0/%d0%a7%d0%b8%d1%81%d0%bb%d0%be-%d0%bf%d1%80%d0%b8%d0%b1%d1%8b%d1%82%d0%b8%d0%b9

In [ ]:
df['flow'] = df.country
df['flow'] = df['flow'].replace('GreatBritain', 36316000)
df['flow'] = df['flow'].replace('France', 89322000)
df['flow'] = df['flow'].replace('Spain', 82773000)
df['flow'] = df['flow'].replace('Spain', 82773000)
df['flow'] = df['flow'].replace('Germany', 38881000)
df['flow'] = df['flow'].replace('Italy', 61567200)
df['flow'] = df['flow'].replace('Italy', 61567200)
df['flow'] = df['flow'].replace('CzechRepublic', 10611000)
df['flow'] = df['flow'].replace('Portugal', 16186000)
df['flow'] = df['flow'].replace('Austria', 30816000)
df['flow'] = df['flow'].replace('Netherlands', 18780000)
df['flow'] = df['flow'].replace('Belgium', 9119000)
df['flow'] = df['flow'].replace('Germany', 38881000 )
df['flow'] = df['flow'].replace('Germany', 38881000)
df['flow'] = df['flow'].replace('France', 89322000 )
df['flow'] = df['flow'].replace('Sweden', 7440000)
df['flow'] = df['flow'].replace('Hungary', 17552000)
df['flow'] = df['flow'].replace('Poland', 19622000)
df['flow'] = df['flow'].replace('Irland', 10926000)
df['flow'] = df['flow'].replace('Denmark', 12749000)
df['flow'] = df['flow'].replace('Greece', 30123000)
df['flow'] = df['flow'].replace('Scotland', 36316000 )
df['flow'] = df['flow'].replace('Switzeland', 10362000)
df['flow'] = df['flow'].replace('Portugal', 16186000 )
df['flow'] = df['flow'].replace('Switzeland', 10362000)
df['flow'] = df['flow'].replace('Poland', 19622000)
df['flow'] = df['flow'].replace('Norway', 5688000)
df['flow'] = df['flow'].replace('Finland', 3224000 )
df['flow'] = df['flow'].replace('Slovakia', 2256000)
df['flow'] = df['flow'].replace('Luxembourg', 1018000 )
df['flow'] = df['flow'].replace('Slovenia', 4425000 )





# Cuisine Style

Проверим столбец с кухнями

In [ ]:
df['cuisine'].isna().value_counts()

Посмотрим образец формы предоставления информации с строках

In [ ]:
df['cuisine'][1]

Подсчитаем сколько различных кухонь в каждом ресторане, собрав их в списки и посчитав их длины

In [ ]:
# сделаем копию данных без пропусков, чтоб можно было подсчитать 
# среднее количество кухонь для заполнения пропусков 
df_cus = df['cuisine'].fillna("'No info'")



In [ ]:
df_cus = df_cus.apply(lambda x: x[1:-1])
df_cus = df_cus.apply(lambda x: x.strip("'''"))

In [ ]:
df_cus = df_cus.apply(lambda x: x.split(','))

Пройдем циклом по столбцу, соберем всё в список и создадим новый признак по количеству представленных кухонь для каждого ресторана. Для пропущенных значений установим значение 1 (длина списка со значением 'No info'), предполагая, что ресторан не делает акцент на кухне, тем самым не предполагает специфику в своей работе 

In [ ]:

cus_list = []
for i in df_cus:
    cus_list.append(len(i))
cus_list

np.mean(cus_list)

In [ ]:
df['cus_number'] = cus_list

Проверим, что ничего не потеряли

In [ ]:
df['cus_number'].isna().value_counts()

Воспользуемся кодом из Baseline, чтоб составить признаки по присутствию кухонь в каждом ресторане.

In [ ]:

df['cuisine'] = df['cuisine'].fillna('""No_info"')

new = pd.DataFrame(df.cuisine.dropna())
a = list(new.cuisine)
b = list()

def l(x):
    i = 0
    for g in x:
        f = x[i].split(',')
        v = 0
        for g in f:
            h = f[v][2:-1].replace("'", '')
            v = +1
            b.append(h)
        i += 1
        
l(a)

from collections import Counter

coun=Counter(b)
coun=dict(coun)
coun=pd.DataFrame({'count':coun}, columns=['count'])
a=coun['count'].mean()

b=list(coun.query('count > @a').index)
b

#оставляем только самые популярные кухни

def find_item(cell):
    if item in cell:
        return 1
    return 0


for item in b:
    df[item] = df['cuisine'].apply(find_item)

df['cuisine'] = df['cuisine'].apply(lambda x: len(x))

# len(df['cuisine'])

# Ranking

Преобразуем значения в рейтинге в INT

In [ ]:
df['ranking'].isna().value_counts()

In [ ]:
df['ranking'] = df['ranking'].apply(lambda x: int(x))

# Price

Смотрим пропуски в данных

In [ ]:
df['price'].isna().value_counts()

Проверим формат данных и наличие пропусков.

In [ ]:
df['price'].unique()

Указанные диапазоны цен переведем в числовой формат. Пропуски заменим средней ценой

In [ ]:
df['price'] = df['price'].replace('$',1)
df['price'] = df['price'].replace('$$ - $$$',2)
df['price'] = df['price'].replace('$$$$',3)

Пропуски заполним средним значением цены

In [ ]:
price_mean = round(df['price'].mean(),2)

In [ ]:
df['price'] = df['price'].fillna(price_mean)

In [ ]:
df['price'].unique()

Проверим количество строк

In [ ]:
df['price'].isna().value_counts()


# Reviews

In [ ]:
# df.head()

Проверим данные. Обратим внимание, что пропуски здесь отражены, как пустые квадратные скобки [[], []].

In [ ]:
df['reviews'].unique()

In [ ]:
df['reviews'] = df.reviews.replace('[[], []]', 'No_info')

Из отзывов выделим даты последних отзывов, чтоб соотнести их с нынешней датой, для определения давности последнего

In [ ]:
df['last_rew'] = df['reviews']

df['last_rew']=df['last_rew'].str[-27:-17]

now = datetime.now()

#base['Last_rew'][base.Last_rew.str.contains("]")]=now
df['last_rew'][df.last_rew.str.contains("]")==True] = now
df['last_rew'] = df['last_rew'].fillna(now)

# приравниваем строки без даты к сегодня

df['last_rew'] = [pd.to_datetime(i) for i in df.last_rew]

In [ ]:
df['last_rew_date'] = abs(df['last_rew']-now)
df['last_rew_date'] = [i.total_seconds() for i in df.last_rew_date]
df['last_rew_date'] = df['last_rew_date']
df['last_rew_date'] = df['last_rew_date'].fillna(0)



# print('')
# print(len(df.last_rew_date))
print((df.last_rew_date[1]))

# Number of reviews

In [ ]:
df['num_rew'].isna().value_counts()

Видим пропуски в данных. Пропущенные значения в количестве отзывов, но, фактически, имеющие таковые заменим на среднее значение

In [ ]:
mean_rev = df['num_rew'].mean()
mean_rev

In [ ]:
df['num_rew'] = df['num_rew'].fillna('No_info')

Сравним фактическое отсутвие информации с отсутвием/наличием отзывов в колонке reviews

In [ ]:
df_rev = df[(df['num_rew'] == 'No_info') & (df['reviews'] == 'No_info')]
rev_index = list(df_rev.index)


In [ ]:
index_list = []
for i in df_rev.index:
    index_list.append(i)

for i in index_list:
    df['num_rew'][i] = 0
    

Отсутствующие значения заменим средним количеством отзывов

In [ ]:
df['num_rew'] = df['num_rew'].replace('No_info',mean_rev)

In [ ]:
type(np.array(df['num_rew']))

In [ ]:
df['num_rew'].isna().value_counts()

In [ ]:
df['population'].isna().value_counts()

Добавим признак соотношения населения к количеству отзывов

In [ ]:
ratio_list = []
for i in range(0, len(df.num_rew)):
    ratio_list.append(df['num_rew'][i]/df['population'][i])



In [ ]:
df['ratio_rev'] = ratio_list

# ID_TA

Иднетификатор с системе Trip Advisor. Переведем в INT

In [ ]:
df['id_ta'] = df['id_ta'].str[1:]

In [ ]:
for i in df.id_ta:
    i = int(i)

In [ ]:
# df.head()

# Копия сета для визуализации

In [ ]:
data_vis = df

# Дополнительные шаги для оптимизации модели

Проверим, может в датасете есть сетевые рестораны и, если есть, добавим для них идентификатор

In [ ]:
net = df.id.value_counts()
net_dict = dict(net)

net_frame = pd.DataFrame({'count':net_dict}, columns = ['count'])

id_index = net_frame.index
id_key = net_frame.values

df['net'] = df['id'].replace(id_index,id_key)

# df.head()
    

In [ ]:
df.loc[(df['net'] == 1), 'count'] = 0

In [ ]:
df.loc[(df['net'] > 1), 'count'] = 1

In [ ]:
df['count'].value_counts()

Используем значение соотношения общего турпотока к населению города, для примерной оценки приезжих по странам 

In [ ]:
data = df
df['flow_pop'] = df['flow']/df['population']
stand = np.std(df['flow_pop'])
mean = np.mean(df['flow_pop'])
df['std_flow'] = (df['flow_pop'] - mean)/stand



Очистим данные, убрав ненужные нечисловые признаки

In [ ]:
df = df.drop('id',axis = 1)
df = df.drop('city',axis = 1)
df = df.drop('url',axis = 1)
df = df.drop('country',axis = 1)
df = df.drop('last_rew',axis = 1)
df = df.drop('reviews',axis = 1)
df = df.drop('cuisine',axis = 1)
# df = df.drop('population',axis = 1)
# df = df.drop('flow_pop',axis = 1)
# df = df.drop('flow',axis = 1)
# df = df.drop('count',axis = 1)
# df = df.drop('No_info',axis = 1)
# df = df.drop('Europian',axis = 1)
# df = df.drop('cus_number',axis = 1)
# df = df.drop('Indian',axis = 1)
# df = df.drop('French',axis = 1)

# Визуализация признаков


Смотрим распределение городских рейтингов в тренировочном наборею Окровенно плохих значительно меньше 

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)
df_train['Ranking'].describe()


In [ ]:
boxplot = df_train.boxplot(column = ['Ranking'])

Проверим значения Rankings на выбросы.

In [ ]:
# median = df_train.Ranking.median()
# IQR = df_train.Ranking.quantile(0.75) - df_train.Ranking.quantile(0.25)
# perc25 = df_train.Ranking.quantile(0.25)
# perc75 = df_train.Ranking.quantile(0.75)
# print('25-й перцентиль: {},'.format(perc25), '75-й перцентиль: {},'.format(perc75),
#       "IQR: {}, ".format(IQR), "Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR))

Согласно расчёту значения внутреннего рейтинга выше 11690 статистически являются выбросами.

Уберем рестораны, являющиеся "выбросами" из общего датасета

In [ ]:
# df = df[df['ranking'] <= 11690]


Распределение по городам очевидно показывает преобладание в больших городах

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

Распределение внутренних рейтингов в Лондоне. 

Распределение рейтингов по городам. Очевидно, что города с большим населением имеют больше высоких оценок. Целесообразно для оценки использовать соотношение рейтинга к населению

In [ ]:
data_vis.columns

In [ ]:
for x in (data_vis['city'].value_counts())[0:10].index:
    data_vis['ranking'][data_vis['city'] == x].hist(bins=100)
plt.show()

Распределение отзывов для рейтинга 5. 

Преобладающее большинство ресторанов с рейтингом 5 имеют относительно небольшое количество отзывов. Из всего тренеровочного сета таких ресторанов 3879

In [ ]:
df['ranking'][df['rating'] == 5].hist(bins=100)

In [ ]:
df_high = df_train[df_train['Rating'] == 5]
df_high.Rating.value_counts()

Распределение отзывов для рейтинга ниже 4

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

In [ ]:
df.rating.isna().value_counts()

In [ ]:
corr_matrix = df.corr(method = 'pearson')
# corr_matrix

In [ ]:
corr_matrix.rating.sort_values(ascending = False)

Согласно корреляционной матрице наибольшее влияние оказывает сетевой формат ресторана. Очевидно, что благодаря охвату. Сам по себе сетевой признак (1 или 0) оказывает меньшее влияние.

In [ ]:
corr_matrix.rating.sort_values(ascending = False)[1:]

In [ ]:
plt.figure(figsize=(30,30)) # размер графика
sns.heatmap(data = corr_matrix, annot=True)

In [ ]:
train_data = df.query('mark == 1').drop(['mark'], axis=1)
test_data = df.query('mark == 0').drop(['mark'], axis=1)

y = train_data.rating.values            # наш таргет
X = train_data.drop(['rating'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(regr.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

Ниже указанны шаги, предпринятые для оптимизации модели. Изменения вносились в код выше.

1. Первое приближение (MAE: 0.208358125)

1.1 Согласно оценке в первом приближении наименьшее влияние на прогноз оказали кухни. Вполне ожидаемо город расположения имеет сильное влияние. Попробуем заменить идентификатор города на соотношение отзывов и населения

1.2 Признак Ranking очевидно сильно влияет, так как выступает в роли привлекательности ресторана.

2. Второе приближение. (MAE: 0.20872625) 

2.1 Добавим признак сетевых ресторанов. 

3. Третье приближение MAE: (0.20782499)

3.1 Попробуем убрать незначительные признаки (убираем идентификаторы кухонь), оставив только более или менее выраженные

4. Четвертое приближение (MAE: 0.203624375)

4.1 Добавим нормализацию признака "рейтинг"

5. Пятое приближение (MAE: 0.20099937 )

Со всеми числовыми признаками

# Submission


In [ ]:
# test_data.sample(10)
# test_data = test_data.drop(['rating'], axis=1)

In [ ]:
# len(test_data)

In [ ]:
# len(sample_submission)

In [ ]:
# predict_submission = regr.predict(test_data)

# len(predict_submission)

In [ ]:
# def round_nearest(x, a):
#     return round(x / a) * a

# sample_submission['Rating'] = predict_submission.round(1)
# sample_submission['Rating'] = round_nearest(sample_submission['Rating'], 0.5)


# sample_submission.head(10)

# sample_submission.to_csv('submission.csv', index=False)

In [ ]:
# data = pd.read_csv('submission.csv')
# data.head()